In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
sample_submission = pd.read_csv('./data/sample_submission.csv')

In [3]:
train.head()

,id,document,label
0,1,영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐,0
1,2,히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯...,1
2,3,괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~,1
3,4,아무래도 20년도지난작품이라 지금보기는너무유치하다,0
4,5,지금까지의 영화들이 그랬듯. 이 영화역시 일본에 대한 미화는 여전하다.,0


# I. 전처리

## 1. 데이터의 특징 살펴보기



### 1) 데이터의 양
    train 5,000행과 test 5,000행밖에 없습니다.
    데이터가 굉장히 적은 축에 속합니다.    
    이럴 때에는 모델링 과정에서 Cross Validation Score로 검증합니다.
    이 과정에서, train과 test의 비율은 어느정도 일정하니 Stratify 는 사용하지 않겠습니다.

In [4]:
train.shape, test.shape

((5000, 3), (5000, 2))

In [5]:
train['label'].value_counts()

0    2564
1    2436
Name: label, dtype: int64

In [6]:
from sklearn.model_selection import KFold

### 2) 데이터의 형태
    데이터에 굉장히 많은 특수문자가 있습니다. => 정규표현식을 이용해 제거
    조사는 데이터 분석에 악영향을 끼칠 것으로 보입니다 => 불용어 처리

In [7]:
def return_clean_text(x) : 
    import re
    x = re.sub('[^ ㄱ-ㅣ가-힣+]', ' ', x) # ㄱ에서 ㅣ 그리고 가-힣 이외의 단어는 띄어쓰기로 변호나
    x = re.sub(' +', ' ', x) # ...같은 단어는 띄어쓰기x3 으로 바뀌기 때문에, 하나로 병합
    return x.strip()
train['new_doc'] = train['document'].apply(return_clean_text)
test['new_doc'] = test['document'].apply(return_clean_text)

for i in range(3) : 
    print(train['document'][i], '==>', train['new_doc'][i])

영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐 ==> 영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐
히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯... ==> 히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯
괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~ ==> 괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요


In [8]:
stop_words = ['은','는','이','가','을','를','다','나','까','요']

def return_clean_text2(x) : 
    new_txt = []
    for word in x.split() : 
        try : 
            while word[-1] in stop_words : 
                word = word[:-1]
            new_txt += [word]
        except : pass;
        
    new_txt = ' '.join(new_txt)
    return new_txt

train['clean_doc'] = train['new_doc'].apply(return_clean_text2)
test['clean_doc'] = test['new_doc'].apply(return_clean_text2)

In [9]:
for i in range(3) : 
    print(train['document'][i], '==>', train['new_doc'][i], '==>' , train['clean_doc'][i])

영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐 ==> 영상이나 음악이 이쁘다 해도 미화시킨 불륜일뿐 ==> 영상 음악 이쁘 해도 미화시킨 불륜일뿐
히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯... ==> 히치콕이 이 영화를 봤다면 분명 박수를 쳤을듯 ==> 히치콕 영화 봤다면 분명 박수 쳤을듯
괜찮은 음악영화가 또 나왔군요!!! 따뜻한 겨울이 될 것 같아요~ ==> 괜찮은 음악영화가 또 나왔군요 따뜻한 겨울이 될 것 같아요 ==> 괜찮 음악영화 또 나왔군 따뜻한 겨울 될 것 같아


# Count Vectorizer(TF-Vectorizer)

    Count Vectorizer는 단순하게 Token 에 대한 "빈도수"를 계산합니다.
    해당 단어가 몇번이 나왔는지에 대한 "횟수"를 의미합니다.
    따라서 Count Vectorizer를 이용할 때는 유의할 것과 설정할 것들이 몇가지 있습니다.
    
    1. N-gram
        - N-gram이라는 것은 몇개의 단어를 이어 붙여 하나의 Token으로 볼지에 대한 개념입니다.
        - Example) 나는 오늘 밥을 먹었다. , 띄어쓰기 단위로 분류
            - uni-gram(N=1) : "나는" , "오늘" , "밥을" , "먹었다"
            - bi-gram (N=2) : "나는 오늘", "오늘 밥을", "밥을 먹었다"
            - etc...
        - 특징
            - 개별 단어만 가지고는 알지 못하는 특성을 계산합니다.
            - example) "좋다" 와 "안 좋다"는 완전 별개의 개념인데, 이를 잡아냅니다
        - 문제점
            - 단어의 수가 늘어납니다.
            - 기본적으로 단어가 K개 있다면, kCn 개 만큼 생겨납니다. (n 은 gram 설정 값)
            - 그에 따른 Feature가 증가하므로, 계산량이 증폭됩니다.
        - 선택
            - 이번 test에서는 n_gram 범주를 (1,2)로 설정하여, 개별 token + bigram token 가지 봅니다.

    2. 이상치 제어
        - 기존 문제점
            - 굉장히 많은 빈도로 등장하는 단어들이 있을 수 있습니다.
            - Example) IMDB의 특성상 "영화"라는 단어는 굉장히 많이 나옵니다.
        - 선택
            - 데이터가 적어 별도의 설정은 하지 않습니다.
    

    3. Vectorizer Dimmension 설정
        - 기존 문제점
            - 모든 단어에 대한 빈도수를 측정하다보니, 모든 개별 단어를 하나의 차원으로 만듭니다.
            - 이는 "차원의 저주"로 이어질 수 있습니다.
                - 이를 해결 하기 위해서는 다양한 방법이 있지만, 여기서는 다루지 않겠습니다.
        - 선택
            - 저는 1만, 2만, 3만 단어에 대해서 간단한 MLP 모델의 Validation 점수를 비교하고, 최고의 점수를 가지는 모델을 선정하겠습니다.


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score
import torch
from torch import nn, optim

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [11]:
class mlp(nn.Module) : 

    def __init__(self, max_f) : 
        super().__init__()

        self.main = nn.Sequential(
            nn.Linear(max_f, 1024),
            nn.ReLU(),
            nn.Linear(1024,256),
            nn.ReLU(),
            nn.Linear(256,64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16,4),
            nn.ReLU(),
            nn.Linear(4,1),
            nn.Sigmoid()
        )

    def forward(self, input) : 
        output = self.main(input)
        return output.view(-1)

In [12]:
def train_model(model, data, criterion, optimizer, num_epochs) : 
    import copy

    train_history = []
    valid_history = []
    best_model_weigths = copy.deepcopy(model.state_dict)
    best_val_acc = 0

    for epoch in range(num_epochs) :
        print('='*20)
        print(f'Epoch {epoch}/{num_epochs-1}')

        for phase in ['train','valid'] : 
            if phase == 'train' : 
                model.train()
            else : 
                model.eval()
            
            running_loss = 0.0
            running_acc = 0

            for inputs, labels in data[phase] : 
                inputs = inputs.to(device)
                labels = labels.to(device)
                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train') : 
                    outputs = model(inputs)
                    print(outputs.shape)
                    loss = criterion(outputs, labels)

                    if phase == 'train' : 
                        loss.backward()
                        optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                running_acc += (outputs == labels).sum()
            epoch_loss = running_loss / len(data[phase])
            epoch_acc = running_acc / len(data[phase]) 
            epoch_acc_str = str(int(round(0.982, 2) * 100))+'%'
            print(f'{phase} Loss: {round(epoch_loss,4)}')
            print(f'{phase} Acc : {epoch_acc_str}')

            if phase == 'train':
                train_history.append([epoch_loss, epoch_acc])
            
            if phase == 'val':
                valid_history.append([epoch_loss, epoch_acc])

            if phase == 'val' and epoch_acc > best_val_acc:
                best_val_loss = epoch_loss
                best_val_acc = epoch_acc
                best_model_weigths = copy.deepcopy(model.state_dict())
        print()

    # load best model weights
    model.load_state_dict(best_model_weigths)

    return model, train_history, valid_history

In [13]:
max_features = [10000, 20000, 30000]

for max_f in max_features : 
    cv = CountVectorizer(ngram_range = (1,2), max_features = max_f)
    cv_train = cv.fit_transform(train['new_doc']).toarray()
    break;

In [14]:
data = {}

data['train'] = [[
                    torch.from_numpy(cv_train[:4000]).float(), 
                    torch.from_numpy(train['label'].values[:4000]).float(), 
                    
                ]]
data['valid'] = [[
                
                    torch.from_numpy(cv_train[4000:]).float(),
                    torch.from_numpy(train['label'].values[4000:]).float(), 
                    
                ]]

In [15]:
model = mlp(max_f).to(device)
loss_func = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr = 0.01)
train_model(model, data, loss_func, optimizer, 10)

Epoch 0/9
torch.Size([4000])
train Loss: 2801.1088
train Acc : 98%
torch.Size([1000])
valid Loss: 701.0533
valid Acc : 98%

Epoch 1/9
torch.Size([4000])
train Loss: 2798.4133
train Acc : 98%
torch.Size([1000])
valid Loss: 700.4146
valid Acc : 98%

Epoch 2/9
torch.Size([4000])
train Loss: 2796.1683
train Acc : 98%
torch.Size([1000])
valid Loss: 699.7957
valid Acc : 98%

Epoch 3/9
torch.Size([4000])
train Loss: 2794.0207
train Acc : 98%
torch.Size([1000])
valid Loss: 699.1997
valid Acc : 98%

Epoch 4/9
torch.Size([4000])
train Loss: 2791.945
train Acc : 98%
torch.Size([1000])
valid Loss: 698.622
valid Acc : 98%

Epoch 5/9
torch.Size([4000])
train Loss: 2789.928
train Acc : 98%
torch.Size([1000])
valid Loss: 698.061
valid Acc : 98%

Epoch 6/9
torch.Size([4000])
train Loss: 2787.9694
train Acc : 98%
torch.Size([1000])
valid Loss: 697.4977
valid Acc : 98%

Epoch 7/9
torch.Size([4000])
train Loss: 2785.9566
train Acc : 98%
torch.Size([1000])
valid Loss: 696.8826
valid Acc : 98%

Epoch 8/9
to

AttributeError: 'function' object has no attribute 'copy'